In [26]:
#Lib
import pandas as pd
from joblib import load
#Clustering
from numpy import unique
from numpy import where
from sklearn.cluster import KMeans
#Zips
import py7zr
#elbow point
%pip install kneed
from kneed import KneeLocator
from sklearn.preprocessing import MinMaxScaler
# plots 
%pip install plotly
import plotly.express as px
import matplotlib.pyplot as plt
#Save plots as .png
%pip install -U kaleido

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [27]:
#Accessing transformed data
with py7zr.SevenZipFile('../../../../data/telecom_churn_me/full_transformed_data.7z', mode='r') as z:
    z.extractall(path='full_transformed_data/')

In [28]:
#Defining target column
target = 'TARGET'
target_value = 1.0

In [29]:
#Reading transformed data
df = pd.read_csv('./full_transformed_data/full_transformated_data.csv')
df

,PARTY_NATIONALITY,BILL_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,OTHER_LINES,STATUS,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,Years_stayed,TARGET
0,-0.459156,-0.682278,2.261594,-0.073181,-0.199016,0.012830,1.868853,-0.413438,-0.097150,-0.437630,0.463925,-0.446350,-0.191567,-0.258991,-0.256628,0
1,-0.410067,-0.933307,-0.474652,-0.086672,-0.372874,0.012830,-0.220822,-0.402058,-0.204611,-0.106452,-0.116617,-0.413453,-0.187495,-0.312506,0.398650,0
2,-0.262800,-0.123599,-0.474652,-0.058417,-0.199016,0.012830,-0.220822,0.006603,-0.206067,0.857268,0.332611,0.059813,0.154315,-0.216404,2.528303,0
3,-0.557334,0.726671,-0.474652,-0.038624,-0.025157,-0.114327,-0.220822,0.287265,-0.348701,-0.414817,-0.267762,-0.446350,-0.192129,0.277326,-0.256628,0
4,-0.557334,0.853040,0.893471,-0.026347,0.496418,-0.114327,-0.220822,1.514276,-0.331090,0.807883,0.055025,-0.446350,0.577716,0.360493,1.381567,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984879,-0.557334,-0.023322,-0.474652,-0.045805,-0.372874,-0.114327,-0.220822,0.282989,-0.208291,0.065293,-0.124241,-0.422977,-0.192129,0.181717,-0.256628,0
984880,-0.557334,-0.349816,-0.474652,-0.090196,-0.372874,-0.114327,-0.220822,0.158733,0.273135,-0.225313,-0.204229,-0.446350,-0.192129,-0.229954,-0.420448,0
984881,2.044388,-0.609373,-0.474652,-0.095470,-0.372874,-0.114327,-0.220822,-0.436037,-0.425388,-0.445148,-0.284759,2.446198,-0.190444,-0.174369,-0.256628,0
984882,-0.557334,-0.120565,0.893471,-0.044964,-0.025157,0.139987,-0.220822,-0.430186,-0.422344,-0.445148,-0.300507,-0.446350,-0.192129,-0.058366,-0.092809,0


In [30]:
#Preparing dataset for clustering
y = df[target]
x = df.drop([target], axis=1)
x

,PARTY_NATIONALITY,BILL_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,OTHER_LINES,STATUS,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,Years_stayed
0,-0.459156,-0.682278,2.261594,-0.073181,-0.199016,0.012830,1.868853,-0.413438,-0.097150,-0.437630,0.463925,-0.446350,-0.191567,-0.258991,-0.256628
1,-0.410067,-0.933307,-0.474652,-0.086672,-0.372874,0.012830,-0.220822,-0.402058,-0.204611,-0.106452,-0.116617,-0.413453,-0.187495,-0.312506,0.398650
2,-0.262800,-0.123599,-0.474652,-0.058417,-0.199016,0.012830,-0.220822,0.006603,-0.206067,0.857268,0.332611,0.059813,0.154315,-0.216404,2.528303
3,-0.557334,0.726671,-0.474652,-0.038624,-0.025157,-0.114327,-0.220822,0.287265,-0.348701,-0.414817,-0.267762,-0.446350,-0.192129,0.277326,-0.256628
4,-0.557334,0.853040,0.893471,-0.026347,0.496418,-0.114327,-0.220822,1.514276,-0.331090,0.807883,0.055025,-0.446350,0.577716,0.360493,1.381567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984879,-0.557334,-0.023322,-0.474652,-0.045805,-0.372874,-0.114327,-0.220822,0.282989,-0.208291,0.065293,-0.124241,-0.422977,-0.192129,0.181717,-0.256628
984880,-0.557334,-0.349816,-0.474652,-0.090196,-0.372874,-0.114327,-0.220822,0.158733,0.273135,-0.225313,-0.204229,-0.446350,-0.192129,-0.229954,-0.420448
984881,2.044388,-0.609373,-0.474652,-0.095470,-0.372874,-0.114327,-0.220822,-0.436037,-0.425388,-0.445148,-0.284759,2.446198,-0.190444,-0.174369,-0.256628
984882,-0.557334,-0.120565,0.893471,-0.044964,-0.025157,0.139987,-0.220822,-0.430186,-0.422344,-0.445148,-0.300507,-0.446350,-0.192129,-0.058366,-0.092809


In [31]:
#Getting number of k-means clusters
def elbow(dataset):
    scaler = MinMaxScaler()
    scaler.fit(dataset)
    X=scaler.transform(dataset)
    inertia = []
    max_clusters = dataset.shape[1] + 1 #number of variables
    cluster_number = list(range(1, max_clusters))
    for i in cluster_number:
        kmeans = KMeans(
            n_clusters=i, init="k-means++",
            n_init=10,
            tol=1e-04, random_state=42
        )
        kmeans.fit(X)
        inertia.append(kmeans.inertia_)
    #locating elbow point
    kneedle = KneeLocator(cluster_number, inertia, S=1.0, curve="convex", direction="decreasing")
    clusters_number = kneedle.knee
    if (cluster_number != int):
        kneedle = KneeLocator(cluster_number, inertia, S=0.0, curve="convex", direction="decreasing")
        clusters_number = kneedle.knee
    return clusters_number

In [32]:
clusters_number = elbow(x)
clusters_number

5

In [33]:
#Clustering function
def clustering(dataframe, clusters_number):
    dataframe.reset_index
    #k-means algorithm
    kmeans = KMeans(
            n_clusters=clusters_number, #number of clusters for general dataset
            init="k-means++",
            n_init=10,
            tol=1e-04, 
            random_state=42
        )
    #fitting the algorithm
    kmeans.fit(dataframe)
    #labeling
    clusters=pd.DataFrame(dataframe)
    clusters['label']=kmeans.labels_
    return clusters

In [34]:
#Transformed dataframe labeled 
df = pd.DataFrame(clustering(x, clusters_number))
#Main clustering polar graph
polar=df.groupby("label").mean().reset_index()
polar=pd.melt(polar,id_vars=["label"])
fig0 = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1400)
fig0.show()
fig0.write_image(f'main_cluster_img.png')
#Main clustering pie plot to see clustering distribution
pie0=df.groupby('label').size().reset_index()
pie0.columns=['label','value']
pie0 = px.pie(pie0,values='value',names='label')
pie0.show()
pie0.write_image(f'main_cluster_distribution.png')


/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future ve

In [35]:
#Main dataframe clustered
df.to_csv(f'./main_cluster.csv') #creating filtered cvs 
df

,PARTY_NATIONALITY,BILL_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,OTHER_LINES,STATUS,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,Years_stayed,label
0,-0.459156,-0.682278,2.261594,-0.073181,-0.199016,0.012830,1.868853,-0.413438,-0.097150,-0.437630,0.463925,-0.446350,-0.191567,-0.258991,-0.256628,2
1,-0.410067,-0.933307,-0.474652,-0.086672,-0.372874,0.012830,-0.220822,-0.402058,-0.204611,-0.106452,-0.116617,-0.413453,-0.187495,-0.312506,0.398650,0
2,-0.262800,-0.123599,-0.474652,-0.058417,-0.199016,0.012830,-0.220822,0.006603,-0.206067,0.857268,0.332611,0.059813,0.154315,-0.216404,2.528303,3
3,-0.557334,0.726671,-0.474652,-0.038624,-0.025157,-0.114327,-0.220822,0.287265,-0.348701,-0.414817,-0.267762,-0.446350,-0.192129,0.277326,-0.256628,0
4,-0.557334,0.853040,0.893471,-0.026347,0.496418,-0.114327,-0.220822,1.514276,-0.331090,0.807883,0.055025,-0.446350,0.577716,0.360493,1.381567,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984879,-0.557334,-0.023322,-0.474652,-0.045805,-0.372874,-0.114327,-0.220822,0.282989,-0.208291,0.065293,-0.124241,-0.422977,-0.192129,0.181717,-0.256628,0
984880,-0.557334,-0.349816,-0.474652,-0.090196,-0.372874,-0.114327,-0.220822,0.158733,0.273135,-0.225313,-0.204229,-0.446350,-0.192129,-0.229954,-0.420448,0
984881,2.044388,-0.609373,-0.474652,-0.095470,-0.372874,-0.114327,-0.220822,-0.436037,-0.425388,-0.445148,-0.284759,2.446198,-0.190444,-0.174369,-0.256628,1
984882,-0.557334,-0.120565,0.893471,-0.044964,-0.025157,0.139987,-0.220822,-0.430186,-0.422344,-0.445148,-0.300507,-0.446350,-0.192129,-0.058366,-0.092809,2


In [36]:
#Create a csv focusing in cluster churns
clust = 0 #iterator
df[target] = y
while clust < clusters_number:
    dataframe = pd.DataFrame(df.loc[df['label'] == clust]) #Filter by cluster
    dataframe = dataframe.loc[dataframe[target] == target_value] #Filter by positive targets
    dataframe = dataframe.drop(columns=[target,'label'])
    dataframe.to_csv(f'./cluster{clust}.csv') #creating filtered cvs 
    clust = clust + 1 #iterator
dataframe

,PARTY_NATIONALITY,BILL_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,OTHER_LINES,STATUS,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,Years_stayed
115,-0.557334,0.272237,-0.474652,0.002707,-0.025157,-0.114327,-0.220822,0.079557,0.972979,-0.294789,-0.124574,-0.445039,-0.188759,0.926489,-0.584267
328,-0.557334,-0.054892,-0.474652,-0.057287,-0.372874,0.012830,-0.220822,0.645084,0.743923,0.622009,-0.087288,-0.396502,-0.150140,0.373157,-0.420448
703,-0.410067,-0.160415,0.893471,-0.084624,-0.372874,-0.114327,-0.220822,0.084936,0.629655,1.119618,1.495403,0.989352,0.635012,-0.120073,-0.911906
1217,-0.557334,0.131757,-0.474652,-0.079089,0.148701,-0.114327,-0.220822,0.563834,-0.397389,1.681907,-0.239016,-0.446350,-0.192129,0.226705,-0.748087
1412,-0.017354,0.269212,-0.474652,-0.065866,0.148701,0.012830,-0.220822,0.686133,-0.329434,0.922467,-0.140197,-0.213496,-0.191708,-0.034773,-0.911906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984090,-0.557334,0.348499,-0.474652,-0.038021,-0.199016,0.012830,-0.220822,1.076239,-0.340506,0.902116,-0.225185,-0.193181,-0.192129,-0.042656,-0.911906
984160,-0.066443,0.335480,0.893471,-0.024685,1.191851,-0.114327,-0.220822,2.369936,0.012134,-0.076769,-0.195355,-0.437743,-0.192129,-0.146054,-0.748087
984590,-0.557334,-0.152405,0.893471,-0.036965,-0.199016,-0.114327,-0.220822,-0.020399,2.461964,0.002039,1.328385,-0.446350,-0.192129,0.036425,-0.748087
984689,-0.459156,1.232693,-0.474652,-0.045820,-0.372874,-0.114327,-0.220822,1.395898,0.985439,0.211245,0.372022,-0.279682,-0.065600,0.148225,1.053928


In [37]:
clust = 0 #iterator
while clust < clusters_number:
    sub_cluster = pd.read_csv(f'./cluster{clust}.csv') #read every subcluster .csv
    sub_cluster = sub_cluster.drop(columns=['Unnamed: 0']) #drop the added column
    sub_clusters_number = elbow(sub_cluster) #calculate elbow point for every sub-cluster
    clusters = pd.DataFrame(clustering(sub_cluster, sub_clusters_number)) #k-means clustering method
    #polar sub-clusters graph
    sub_polar=clusters.groupby("label").mean().reset_index() 
    sub_polar=pd.melt(sub_polar,id_vars=["label"])
    fig = px.line_polar(sub_polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1400)
    fig.show() #print here
    fig.write_image(f'cluster{clust}img.png') #save as .png file
    #Pie plot to see sub-cluster's distribution
    pie=clusters.groupby('label').size().reset_index()
    pie.columns=['label','value']
    pie = px.pie(pie,values='value',names='label')
    pie.show() #print here
    pie.write_image(f'cluster{clust}distribution.png') #save as .png file
    clust = clust + 1
    
    

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future ve

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future ve

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future ve